Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [2]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [3]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

notMNIST_large already present - Skipping extraction of notMNIST_large.tar.gz.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
notMNIST_small already present - Skipping extraction of notMNIST_small.tar.gz.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [4]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  image_index = 0
  print(folder)
  for image in os.listdir(folder):
    image_file = os.path.join(folder, image)
    try:
      image_data = (ndimage.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[image_index, :, :] = image_data
      image_index += 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

notMNIST_large/A.pickle already present - Skipping pickling.
notMNIST_large/B.pickle already present - Skipping pickling.
notMNIST_large/C.pickle already present - Skipping pickling.
notMNIST_large/D.pickle already present - Skipping pickling.
notMNIST_large/E.pickle already present - Skipping pickling.
notMNIST_large/F.pickle already present - Skipping pickling.
notMNIST_large/G.pickle already present - Skipping pickling.
notMNIST_large/H.pickle already present - Skipping pickling.
notMNIST_large/I.pickle already present - Skipping pickling.
notMNIST_large/J.pickle already present - Skipping pickling.
notMNIST_small/A.pickle already present - Skipping pickling.
notMNIST_small/B.pickle already present - Skipping pickling.
notMNIST_small/C.pickle already present - Skipping pickling.
notMNIST_small/D.pickle already present - Skipping pickling.
notMNIST_small/E.pickle already present - Skipping pickling.
notMNIST_small/F.pickle already present - Skipping pickling.
notMNIST_small/G.pickle 

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [30]:
test_A = pickle.load(open('notMNIST_large/A.pickle', 'rb'))
img = test_A[1000, :, :]

plt.imshow(img)
plt.show()
plt.close()

# Downsample the image.
def block_mean(image, factor):
    # Generate the labels via meshgrid.
    sx, sy = image.shape
    X, Y = np.ogrid[0:sx, 0:sy]
    regions = sy/factor * (X/factor) + Y/factor
    
    # Calculate the mean corresponding to label regions. Reshape image.
    # Note: index=np.unique(regions) will work too, but is inefficient.
    res = ndimage.mean(image, labels=regions, index=np.arange(regions.max() + 1))
    res.shape = (sx/factor, sy/factor)
    return res

# Test with a factor of 4 (since input is 28x28).
new_img = block_mean(img, 4)
plt.imshow(new_img)
plt.show()
plt.close()

---
Problem 3
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune `train_size` as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [5]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class

  for label, pickle_file in enumerate(pickle_files): 
        
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
        
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Training: (200000, 28, 28) (200000,)
Validation: (10000, 28, 28) (10000,)
Testing: (10000, 28, 28) (10000,)


Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [6]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

In [15]:
plt.figure(1)
plt.hist(train_labels, 10, normed=1, facecolor='b')
plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.title('Train Labels Class Distribution')
plt.show()

plt.figure(2)
plt.hist(test_labels, 10, normed=1, facecolor='b')
plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.title('Test Labels Class Distribution')
plt.show()

#------ Alternative test ------
from collections import Counter
Counter(train_labels), Counter(test_labels)

(Counter({0: 20000,
          1: 20000,
          2: 20000,
          3: 20000,
          4: 20000,
          5: 20000,
          6: 20000,
          7: 20000,
          8: 20000,
          9: 20000}),
 Counter({0: 1000,
          1: 1000,
          2: 1000,
          3: 1000,
          4: 1000,
          5: 1000,
          6: 1000,
          7: 1000,
          8: 1000,
          9: 1000}))

---
Problem 4
---------
Convince yourself that the data is still good after shuffling!

---

In [16]:
#------ Test that the data is still the same ------
# Mean should be roughly 0, and standard deviation 0.5

# Train dataset
print('Train mean = {0}'.format(np.mean(train_dataset),))
print('Train std = {0}'.format(np.std(train_dataset),))

# Test dataset
print('Test mean = {0}'.format(np.mean(test_dataset),))
print('Test std = {0}'.format(np.std(test_dataset),))

Train mean = -0.0813079550862
Train std = 0.454282045364
Test mean = -0.0739829689264
Test std = 0.458816617727


Finally, let's save the data for later reuse:

In [17]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [18]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 690800441


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [8]:
#------ Part 1: Measure the overlap between training, validation and test datasets ------
# Option 1: Look at co-variance matrix of image pairs. WARNING: This is very slow.
# Option 2: Use Mean Squared Error (MSE). This allows us to combine Parts 1 and 2.
# Option 3: Flatten each (28, 28) image into (784,). Keep lists in a nested list. 
# Use set() to remove duplicates.

# Methods: MSE sv SSIM (Structural Similarity Index)
# http://www.pyimagesearch.com/2014/09/15/python-compare-two-images/

# How to compare two similar photos:
# http://blog.iconfinder.com/detecting-duplicate-images-using-python/

# --------------------
# Step 1 - Downsample the images to something more manageable. Use a factor
# of 4 since input images are (28, 28).

def block_mean(image, factor):
    # Generate the labels via meshgrid.
    sx, sy = image.shape
    X, Y = np.ogrid[0:sx, 0:sy]
    regions = sy/factor * (X/factor) + Y/factor
    
    # Calculate the mean corresponding to label regions. Reshape image.
    # Note: index=np.unique(regions) will work too, but is inefficient.
    res = ndimage.mean(image, labels=regions, index=np.arange(regions.max() + 1))
    res.shape = (sx/factor, sy/factor)
    return res

factor = 4

# Initialize arrays for storing downsampled images.
(m_train, n_train, k_train) = np.shape(train_dataset)
(m_valid, n_valid, k_valid) = np.shape(valid_dataset)
(m_test, n_test, k_test) = np.shape(test_dataset)

train_downsampled = np.zeros((m_train, n_train/factor, k_train/factor))
valid_downsampled = np.zeros((m_valid, n_valid/factor, k_valid/factor))
test_downsampled = np.zeros((m_test, n_test/factor, k_test/factor))

# Calculate downsampled images.
for i in np.arange(m_train):
    train_downsampled[i, :, :] = block_mean(train_dataset[i, :, :], factor)

for i in np.arange(m_valid):
    valid_downsampled[i, :, :] = block_mean(valid_dataset[i, :, :], factor)

for i in np.arange(m_test):
    test_downsampled[i, :, :] = block_mean(test_dataset[i, :, :], factor)

# Save the downsampled output.
np.savez('downsampled_images', train_downsampled=train_downsampled, 
        valid_downsampled=valid_downsampled, test_downsampled=test_downsampled)

print('Saved downsampled data...')

Saved downsampled data...


In [7]:
import time
import hashlib

# Step 2 - Calculate Mean Squared Error (MSE).

# We use the MSE because that enables removal of exact (MSE = 0) and near
# duplicates (MSE < threshold). We need to flatten and downsample each image into 
# the same size vector arrays first, because even MSE takes way too long to compare 
# each element of the valid/test datasets to the train dataset.

data = np.load('downsampled_images.npz')

train_downsampled = data['train_downsampled']
valid_downsampled = data['valid_downsampled']
test_downsampled = data['test_downsampled']

# Note: Comparing each element to every other element in the dataset is O(N^2).
# This is impractical for large datasets. We will do image hashing on the
# downsampled dataset instead. Note also that we are finding "similar" images
# by comparing hashes on the downsampled images. We could go one step further
# by comparing the Hamming distances on the resultant hashes.

# Sources:
# https://www.safaribooksonline.com/blog/2013/11/26/image-hashing-with-python/
# https://discussions.udacity.com/t/assignment-1-problem-5/45657/18
# https://en.wikipedia.org/wiki/Hamming_distance

t1 = time.time()

train_hashes = [hashlib.sha1(x).digest() for x in train_downsampled]
valid_hashes = [hashlib.sha1(x).digest() for x in valid_downsampled]
test_hashes = [hashlib.sha1(x).digest() for x in test_downsampled]

valid_in_train = np.in1d(valid_hashes, train_hashes)
test_in_train = np.in1d(test_hashes, train_hashes)
test_in_valid = np.in1d(test_hashes, valid_hashes)

valid_index = ~valid_in_train
test_index = ~(test_in_train | test_in_valid)

valid_dataset_clean = valid_dataset[valid_index]
valid_labels_clean = valid_labels[valid_index]

test_dataset_clean = test_dataset[test_index]
test_labels_clean = test_labels[test_index]

t2 = time.time()

# Print results
print('Time elapsed = %0.2fs' % (t2-t1))
print('valid in train = %d' % np.sum(valid_in_train))
print('test in train = %d' % np.sum(test_in_train))
print('test in valid = %d' % np.sum(test_in_valid))

Time elapsed = 0.84s
valid in train = 1068
test in train = 1326
test in valid = 200


---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [8]:
# Load randomnized data.
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    data = pickle.load(f)

train_dataset = data['train_dataset']
train_labels = data['train_labels']
valid_dataset = data['valid_dataset']
valid_labels = data['valid_labels']
test_dataset = data['test_dataset']
test_labels = data['test_labels']

# ------------------------------------
# Step 1 - Convert training data from 3D to 2D for the classifier.
(m_train, n_train, k_train) = np.shape(train_dataset)
(m_valid, n_valid, k_valid) = np.shape(valid_dataset)
(m_test, n_test, k_test) = np.shape(test_dataset)

# Model hyperparameters.
num_samples = 5000
C = 1.0

# Reshape the data into 2D dataset.
X_train = np.reshape(train_dataset, (m_train, n_train*k_train))[0:num_samples]
X_valid = np.reshape(valid_dataset, (m_valid, n_valid*k_valid))[0:num_samples]
X_test = np.reshape(test_dataset, (m_test, n_test*k_test))[0:num_samples]

# Define the labels.
y_train = train_labels[0:num_samples]
y_valid = valid_labels[0:num_samples]
y_test = test_labels[0:num_samples]

# ------------------------------------
# Test various classifiers.

# Define different classifier objects.
classifiers = {'L1 logistic (OvR)': LogisticRegression(C=C, penalty='l1'),
               'L2 logistic (OvR)': LogisticRegression(C=C, penalty='l2'),
               'Linear SVC': SVC(kernel='linear', C=C, probability=True,
                                 random_state=0),
               'L2 logistic (Multinomial)': LogisticRegression(C=C, penalty='l2', 
                solver='lbfgs', multi_class='multinomial'),
              }

# ------------------------------------
print('Training...')

for i, (name, classifier) in enumerate(classifiers.items()):
    
    t1 = time.time()
    classifier.fit(X_train, y_train)
    t2 = time.time()
    
    print('%s training time = %0.2fs' % (name, t2-t1))
    
    # Test on validation data.
    y_pred = classifier.predict(X_valid) # Predict label data.
    y_score = classifier.score(X_valid, y_valid) # Calculate mean accuracy.
    
    print('Mean accuracy = %0.1f' % y_score)
    

Training...
L2 logistic (OvR) training time = 17.73s
Mean accuracy = 0.8
Linear SVC training time = 41.88s
Mean accuracy = 0.8
L1 logistic (OvR) training time = 12.17s
Mean accuracy = 0.8
L2 logistic (Multinomial) training time = 0.85s
Mean accuracy = 0.8
